## QTM 350: Data Science Computing

### Assignment 08: SQL Practice with Python

### Due 18 June 2025

### Instructions:
Use Python's `sqlite3` library to complete the following tasks. Connect to an in-memory SQLite database or create a file named `assignment08.db`.

Ensure all your Python code and the query results (where applicable) are included in this notebook. Submit this `.ipynb` file on Canvas.

Begin by running the setup code cell to create and populate the `drivers` table.

### Setup: Create and Populate Database

In [1]:
import sqlite3
import pandas as pd # Often useful for displaying results

# Connect to the database (or create it)
# conn = sqlite3.connect('assignment08.db') 
conn = sqlite3.connect(':memory:') # Option for in-memory database
cursor = conn.cursor()
print("Database connected.")

# --- Create drivers Table ---
cursor.execute("DROP TABLE IF EXISTS drivers;")
create_table_sql = """
CREATE TABLE drivers(
    driver_id INT PRIMARY KEY,
    driver_name VARCHAR(30),
    nationality VARCHAR(15),
    team VARCHAR(30),
    victories INT,
    podiums INT,
    poles INT,
    points FLOAT
);
"""
cursor.execute(create_table_sql)
print("Table 'drivers' created.")

# --- Insert Data ---
drivers_data = [
    (1, 'Lewis Hamilton', 'British', 'Mercedes', 103, 182, 103, 4160.5),
    (2, 'Michael Schumacher', 'German', 'Ferrari', 91, 155, 68, 1566.0),
    (3, 'Ayrton Senna', 'Brazilian', 'McLaren', 41, 80, 65, 614.0),
    (4, 'Sebastian Vettel', 'German', 'Red Bull', 53, 122, 57, 3097.0),
    (5, 'Niki Lauda', 'Austrian', 'Ferrari', 25, 54, 24, 420.5),
    (6, 'Fernando Alonso', 'Spanish', 'Renault', 32, 101, 22, 2061.0),
    (7, 'Alain Prost', 'French', 'McLaren', 51, 106, 33, 798.5),
    (8, 'Charles Leclerc', 'Monegasque', 'Ferrari', 5, 21, 18, 699.0),
    (9, 'Max Verstappen', 'Dutch', 'Red Bull', 52, 92, 28, 2411.0),
    (10, 'Jenson Button', 'British', 'Brawn GP', 15, 50, 8, 1235.0),
    (11, 'Kimi Raikkonen', 'Finnish', 'Ferrari', 21, 103, 18, 1873.0),
    (12, 'Daniel Ricciardo', 'Australian', 'Red Bull', 8, 32, 3, 1274.0),
    (13, 'Valtteri Bottas', 'Finnish', 'Mercedes', 10, 67, 20, 1731.0),
    (14, 'Rubens Barrichello', 'Brazilian', 'Ferrari', 11, 68, 13, 658.0)
]

insert_sql = "INSERT INTO drivers VALUES (?, ?, ?, ?, ?, ?, ?, ?);"
cursor.executemany(insert_sql, drivers_data)
conn.commit()
print(f"{cursor.rowcount} records inserted into 'drivers'.")

# --- Verify Data Insertion ---
print("\n--- Initial Drivers Data ---")
df_initial = pd.read_sql_query("SELECT * FROM drivers;", conn)
print(df_initial)

Database connected.
Table 'drivers' created.
14 records inserted into 'drivers'.

--- Initial Drivers Data ---
    driver_id         driver_name nationality      team  victories  podiums  \
0           1      Lewis Hamilton     British  Mercedes        103      182   
1           2  Michael Schumacher      German   Ferrari         91      155   
2           3        Ayrton Senna   Brazilian   McLaren         41       80   
3           4    Sebastian Vettel      German  Red Bull         53      122   
4           5          Niki Lauda    Austrian   Ferrari         25       54   
5           6     Fernando Alonso     Spanish   Renault         32      101   
6           7         Alain Prost      French   McLaren         51      106   
7           8     Charles Leclerc  Monegasque   Ferrari          5       21   
8           9      Max Verstappen       Dutch  Red Bull         52       92   
9          10       Jenson Button     British  Brawn GP         15       50   
10         11      K

1) List drivers ordered by `team` alphabetically and then by `victories` in descending order.

In [6]:
query1 = """
SELECT driver_name, team, victories
FROM drivers
ORDER BY team DESC, victories DESC
"""

display(pd.read_sql(query1, conn))

,driver_name,team,victories
0,Fernando Alonso,Renault,32
1,Sebastian Vettel,Red Bull,53
2,Max Verstappen,Red Bull,52
3,Daniel Ricciardo,Red Bull,8
4,Lewis Hamilton,Mercedes,103
5,Valtteri Bottas,Mercedes,10
6,Alain Prost,McLaren,51
7,Ayrton Senna,McLaren,41
8,Michael Schumacher,Ferrari,91
9,Niki Lauda,Ferrari,25


2) List drivers who have won more races than the average number of victories across all drivers.

In [8]:
query2 = """
SELECT driver_name, victories
FROM drivers
WHERE victories > (SELECT AVG(victories) FROM drivers)
"""

display(pd.read_sql(query2, conn))

,driver_name,victories
0,Lewis Hamilton,103
1,Michael Schumacher,91
2,Ayrton Senna,41
3,Sebastian Vettel,53
4,Alain Prost,51
5,Max Verstappen,52


3) Find all drivers with more than 50 podiums OR who drive for the 'Ferrari' team.

In [9]:
query3 = """
SELECT driver_name, team, podiums
FROM drivers
WHERE team IN ('Ferrari') OR podiums > 50
"""

display(pd.read_sql(query3, conn))

,driver_name,team,podiums
0,Lewis Hamilton,Mercedes,182
1,Michael Schumacher,Ferrari,155
2,Ayrton Senna,McLaren,80
3,Sebastian Vettel,Red Bull,122
4,Niki Lauda,Ferrari,54
5,Fernando Alonso,Renault,101
6,Alain Prost,McLaren,106
7,Charles Leclerc,Ferrari,21
8,Max Verstappen,Red Bull,92
9,Kimi Raikkonen,Ferrari,103


4) Count the number of drivers for each `nationality`, but only include drivers who have more than 40 `podiums`. Order the result by count in descending order.

In [24]:
query4 = """
SELECT nationality, COUNT (*) AS num_drivers
FROM drivers
WHERE podiums > 40
GROUP BY nationality
ORDER BY num_drivers DESC
"""

display(pd.read_sql(query4, conn))

,nationality,num_drivers
0,German,2
1,Finnish,2
2,British,2
3,Brazilian,2
4,Spanish,1
5,French,1
6,Dutch,1
7,Austrian,1


5) Calculate the average `points` for each `team`, rounded to two decimal places.

In [68]:
query5 = """
SELECT team, ROUND(AVG(points), 2) AS avg_points_rounded
FROM drivers
GROUP BY team
"""

display(pd.read_sql(query5, conn))

,team,avg_points_rounded
0,Brawn GP,1235.00
1,Ferrari,1043.30
2,McLaren,706.25
3,Mercedes,2945.75
4,Red Bull,2260.67
5,Renault,2061.00


6) List drivers who are in the top 20% based on their `points`.

In [ ]:
# Assuming top 20% of 14 drivers = 2 drivers with the most points:
query6 = """
SELECT driver_name, points
FROM drivers
ORDER BY points DESC
LIMIT 2
"""

display(pd.read_sql(query6, conn))


# Alternatively, assuming top 20% of 14 drivers = drivers who are above the 80th percentile for points:
query6_5 = """
SELECT driver_name, points 
FROM drivers
WHERE points > (SELECT MAX(points) * 0.8 FROM drivers)
"""

display(pd.read_sql(query6_5 , conn))

,driver_name,points
0,Lewis Hamilton,4160.5
1,Sebastian Vettel,3097.0


,driver_name,points
0,Lewis Hamilton,4160.5


7) Calculate the total `victories`, total `podiums`, and average `points` for drivers with 'German' `nationality`.

In [71]:
query7 = """
SELECT nationality,
    SUM(victories) AS num_victories,
    SUM(podiums) AS num_podiums,
    AVG(points) AS avg_points
FROM drivers
WHERE nationality = 'German';
"""

display(pd.read_sql(query7, conn))

,nationality,num_victories,num_podiums,avg_points
0,German,144,277,2331.5


8) List all drivers whose `driver_name` contains both the letter 'a' and the letter 'e' (case-insensitive).

In [58]:
query8 = """
SELECT driver_name
FROM drivers
WHERE driver_name LIKE '%a%' AND driver_name LIKE '%e%';
"""

display(pd.read_sql(query8, conn))

,driver_name
0,Lewis Hamilton
1,Michael Schumacher
2,Ayrton Senna
3,Sebastian Vettel
4,Fernando Alonso
5,Charles Leclerc
6,Max Verstappen
7,Kimi Raikkonen
8,Daniel Ricciardo
9,Valtteri Bottas


9) Retrieve the driver with the most `victories` within each `team`.

In [59]:
query9 = """
SELECT driver_name, team
FROM (SELECT driver_name, team, victories, 
    RANK() OVER (PARTITION BY team ORDER BY victories DESC) AS team_rank
    FROM drivers)
WHERE team_rank = 1;
"""

display(pd.read_sql(query9, conn))

,driver_name,team
0,Jenson Button,Brawn GP
1,Michael Schumacher,Ferrari
2,Alain Prost,McLaren
3,Lewis Hamilton,Mercedes
4,Sebastian Vettel,Red Bull
5,Fernando Alonso,Renault


10) List drivers whose `points` are below the average `points` for their respective `team`.

In [60]:
query10 = """
SELECT driver_name, points, avg_team_points
FROM(SELECT driver_name, team, points, 
    AVG(points) OVER (PARTITION BY team) AS avg_team_points
    FROM drivers)
WHERE points < avg_team_points;
"""

display(pd.read_sql(query10, conn))

,driver_name,points,avg_team_points
0,Niki Lauda,420.5,1043.300000
1,Charles Leclerc,699.0,1043.300000
2,Rubens Barrichello,658.0,1043.300000
3,Ayrton Senna,614.0,706.250000
4,Valtteri Bottas,1731.0,2945.750000
5,Daniel Ricciardo,1274.0,2260.666667


11) Cleanup: Close the database connection

In [72]:
# Close the connection
conn.close()
print("Database connection closed.")

Database connection closed.
